In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from collections import OrderedDict
import matplotlib.pyplot as plt

In [2]:
from ptranking.data.data_utils import LTRDataset, SPLIT_TYPE
data_id = 'MQ2007_Super'
#MQ2007内にあるもの
s1 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S1.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
s2 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S2.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
s3 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S3.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
s4 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S4.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
s5 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S5.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
test_data = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='test.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)

In [ ]:
s1[0][0]

In [3]:
data_list = [s1, s2, s3, s4, s5]

In [4]:
#s1-s5でranking scoreとlabelをlistに分割する
ranking=[]
label=[]
for doc in range(len(data_list)):
    ranking.append([])
    label.append([])
    for query in range(len(data_list[doc])):
        _, batch_ranking, std_label = data_list[doc][query]
        ranking[-1].append(batch_ranking)
        label[-1].append(std_label)

In [5]:
label[0][0]

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [6]:
layers = [nn.Linear(46, 128),
          nn.ReLU(),
          nn.Linear(128, 64),
          nn.ReLU(),
          nn.Linear(64, 32),
          nn.ReLU(),
          nn.Linear(32, 1)]

In [7]:
class RankNet(nn.Module):
    def __init__(self, layers):
        super(RankNet, self).__init__()
        self.model = nn.Sequential(*layers)
        
    def forward(self, batch_ranking, label):
        batch_pred = self.model(batch_ranking) # [1, 40, 1]
        batch_sij = torch.squeeze(batch_pred, 0) - torch.squeeze(batch_pred, 2) # [40, 40]

        label_dim = torch.squeeze(label)
        label_diffs = torch.unsqueeze(label_dim, 1) - label # [40,40]
        batch_Sij = torch.clamp(label_diffs, -1, 1)
                
        # computes p_ij
        batch_pij = 1 / (1 + torch.exp(-batch_sij)) # [40, 40]
                
        # computes loss
        batch_loss = 0.5 * (1-batch_Sij) * batch_sij + torch.log(1+torch.exp(-batch_sij)) #[40, 40]
                
        # make upper triangular matrix
        batch_loss_triu = torch.triu(batch_loss, diagonal=1)

        # computes the mean value of a
        num_elements = (batch_loss_triu.size(1)*batch_loss_triu.size(1)-batch_loss_triu.size(1))/2
        batch_loss_mean = torch.sum(batch_loss_triu)/num_elements
        
        #loss_list.append(batch_loss_mean)
                
        #print("batch_loss_mean", batch_loss_mean)
        
        return batch_loss_mean
    
    def predict(self, x):
        return self.model(x)

In [8]:
def DCG(sorted_labels, cutoff):
    denoms = torch.log2(torch.arange(2, cutoff+2))
    nums = torch.pow(2, sorted_labels[0:cutoff])-1
    dcg = sum(nums / denoms)
    return dcg

In [9]:
def nDCG(ideal, pred, k):
    dcg_f = DCG(pred, k)
    dcg = DCG(ideal, k)
    nDCG = dcg_f / dcg
    return nDCG

In [10]:
def compute_ndcg(rank,label,cutoff):     
    t = model.predict(rank)
    r = torch.argsort(t,dim=1,descending = True)
    ideal,_ = torch.sort(label,1,descending = True)
    ideal = torch.unsqueeze(ideal,2).reshape(-1)
    pred = torch.gather(label.unsqueeze(2),1,r).reshape(-1)
    ndcg = torch.nan_to_num(nDCG(ideal,pred,cutoff))
    return ndcg

In [18]:
def training_loop(n_epochs,optimizer,model,ranking_score,std_label):
    count = 0
    best_ndcg = 0
    
    while count < 5:
        
        print("fold:{0}".format(count+1))
        # 最初の3つをtrainに
        train = 2
        # 1つをvalidationに
        val = 3
        # 最後をtestに
        test = 4
    
        # モデルの訓練
        print("*train*")
        
        for epoch in range(1, n_epochs + 1):
        
            # クエリごとの訓練
            batch_loss = 0
            batch_ndcg = 0
            
            train_ranking = ranking[count]+ranking[(count+1)%5]+ranking[(count+2)%5]
            train_label = label[count]+label[(count+1)%5]+label[(count+2)%5]
            
            for train_query in range(len(train_ranking)):
                batch_loss += model.forward(batch_ranking=train_ranking[train_query], label=train_label[train_query])
                batch_ndcg += compute_ndcg(train_ranking[train_query],train_label[train_query],5)
                    
            fold_loss_mean = batch_loss / len(train_ranking)
            fold_ndcg_mean = batch_ndcg / len(train_ranking)
            
            if epoch ==1 or epoch%10==0:
                print('epoch:{0}, loss:{1}, ndcg:{2}'.format(epoch, fold_loss_mean, fold_ndcg_mean))
            
            # パラメータ選択
            val_loss = 0
            val_ndcg = 0
            for val_query in range(len(ranking[(count+val)%5])):
                val_loss += model.forward(batch_ranking=ranking[(count+val)%5][val_query], label=label[(count+val)%5][val_query])
                val_ndcg += compute_ndcg(ranking[(count+val)%5][val_query],label[(count+val)%5][val_query],5)
                
            val_loss_mean = val_loss / len(ranking[(count+val)%5])
            val_ndcg_mean = val_ndcg / len(ranking[(count+val)%5])
            
            
            if val_ndcg_mean > best_ndcg:
                best_ndcg = val_ndcg_mean
                
                #print('epoch:{0}, loss:{1}, ndcg:{2}'.format(epoch, val_loss_mean, val_ndcg_mean))
            
                # 訓練したモデルの保存
                torch.save(model.state_dict(), 'weight.pth')
            
            optimizer.zero_grad()
            fold_loss_mean.backward()
            optimizer.step()
            
        # パラメータの読み込み
        param = torch.load('weight.pth')
        model.load_state_dict(param)
        
        
        # test(関数を使わないバージョンだよーーーー)
        print("*test*")    
        with torch.no_grad():
            test_ndcg = 0
            test_loss = 0
            for test_query in range(len(ranking[(count+test)%5])):
            #computes ndcg
                test_loss += model.forward(batch_ranking=ranking[(count+test)%5][test_query], label=label[(count+test)%5][test_query])
                test_ndcg += compute_ndcg(ranking[(count+test)%5][test_query],label[(count+test)%5][test_query],5)
                  
            test_ndcg_mean = test_ndcg / len((ranking[(count+test)%5]))
        
        print('loss:{0}, ndcg:{1}'.format(test_loss_mean, test_ndcg))
        
        count += 1
        print("*****************************")

In [19]:
# クラスのインスタンス化
model = RankNet(layers)

In [20]:
training_loop(
    n_epochs = 100,
    optimizer = torch.optim.SGD(model.parameters(),lr = 0.01),
    model = model,
    ranking_score=ranking,
    std_label=label,
    )

fold:1
*train*
epoch:1, loss:0.6932628154754639, ndcg:0.19019044935703278
epoch:10, loss:0.6932562589645386, ndcg:0.19231191277503967
epoch:20, loss:0.6932491660118103, ndcg:0.19418761134147644
epoch:30, loss:0.6932414174079895, ndcg:0.19616520404815674
epoch:40, loss:0.6932332515716553, ndcg:0.19821470975875854
epoch:50, loss:0.6932259202003479, ndcg:0.1997835785150528
epoch:60, loss:0.6932186484336853, ndcg:0.2020106315612793
epoch:70, loss:0.6932104229927063, ndcg:0.20368671417236328
epoch:80, loss:0.693202793598175, ndcg:0.20514008402824402
epoch:90, loss:0.6931951642036438, ndcg:0.20755286514759064
epoch:100, loss:0.6931875348091125, ndcg:0.2104966938495636
*test*
loss:0.6931732296943665, ndcg:0.20646017789840698
*****************************
fold:2
*train*
epoch:1, loss:0.6931903958320618, ndcg:0.2066461145877838
epoch:10, loss:0.6931833624839783, ndcg:0.20945218205451965
epoch:20, loss:0.6931757926940918, ndcg:0.21162128448486328
epoch:30, loss:0.6931681632995605, ndcg:0.2134959